<a href="https://colab.research.google.com/github/Marcux777/Ci-ncia-de-Dados/blob/main/VI%20Desafio%20de%20Ci%C3%AAncia%20de%20dados/Tratamentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40712 sha256=dd3f0de969b6648254155014dc029ea7710e06b687f4b9e201aef5232c6e3152
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


Importando as bibliotecas que usaremos

In [ ]:
import pandas as pd
import math
import numpy as np
import googlemaps as maps
import plotly.express as px
import geopandas as gpd
import folium
from folium import Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

Abrindo o arquivo excel:

In [ ]:
df = pd.read_excel('/content/drive/My Drive/Ciencia de dados/acidentesTransito_cenipa_2010a2021.xlsx', engine='openpyxl')
print(df.head())


  ocorrencia_classificacao  ocorrencia_latitude  ocorrencia_longitude  \
0          INCIDENTE GRAVE        -9.868333e+12         -6.789806e+12   
1                 ACIDENTE        -1.220417e+12         -5.623778e+12   
2                 ACIDENTE        -1.220417e+12         -5.623778e+12   
3                 ACIDENTE        -1.220417e+12         -5.623778e+12   
4                 ACIDENTE        -1.220417e+12         -5.623778e+12   

   ocorrencia_cidade ocorrencia_uf ocorrencia_pais ocorrencia_aerodromo  \
0         RIO BRANCO            AC          BRASIL                 SBRB   
1  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   
2  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   
3  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   
4  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   

  ocorrencia_dia ocorrencia_hora investigacao_aeronave_liberada  ...  \
0     2022-04-14        17:45:00      

Tratamento dos dados dentro do data frame:

In [ ]:
## Buscando e removendo dados duplicados, caso haja
df.drop_duplicates()

# verificando quantas coordenas nulas eu tenho
print(df.isnull().sum())

ocorrencia_classificacao              0
ocorrencia_latitude                 678
ocorrencia_longitude                670
ocorrencia_cidade                     0
ocorrencia_uf                         0
ocorrencia_pais                       0
ocorrencia_aerodromo                  0
ocorrencia_dia                        0
ocorrencia_hora                       0
investigacao_aeronave_liberada     1648
investigacao_status                   0
divulgacao_relatorio_numero          39
divulgacao_relatorio_publicado        0
divulgacao_dia_publicacao             0
total_recomendacoes                   0
total_aeronaves_envolvidas            0
ocorrencia_saida_pista                0
ocorrencia_tipo                       4
ocorrencia_tipo_categoria             4
taxonomia_tipo_icao                   4
recomendacao_numero                   0
recomendacao_dia_assinatura           0
recomendacao_dia_encaminhamento       0
recomendacao_dia_feedback          1137
recomendacao_conteudo                 0


Remover onde a latitude ou longitude são nulas, não quero elas


In [ ]:
df.dropna(subset=['ocorrencia_latitude'], inplace=True)
df.dropna(subset=['ocorrencia_longitude'], inplace=True)
print(df.isnull().sum())

ocorrencia_classificacao              0
ocorrencia_latitude                   0
ocorrencia_longitude                  0
ocorrencia_cidade                     0
ocorrencia_uf                         0
ocorrencia_pais                       0
ocorrencia_aerodromo                  0
ocorrencia_dia                        0
ocorrencia_hora                       0
investigacao_aeronave_liberada     1546
investigacao_status                   0
divulgacao_relatorio_numero          10
divulgacao_relatorio_publicado        0
divulgacao_dia_publicacao             0
total_recomendacoes                   0
total_aeronaves_envolvidas            0
ocorrencia_saida_pista                0
ocorrencia_tipo                       4
ocorrencia_tipo_categoria             4
taxonomia_tipo_icao                   4
recomendacao_numero                   0
recomendacao_dia_assinatura           0
recomendacao_dia_encaminhamento       0
recomendacao_dia_feedback          1106
recomendacao_conteudo                 0


Passar para numerico, caso haja algum valor que não esteja numerico

In [ ]:
df['ocorrencia_latitude'] = pd.to_numeric(df['ocorrencia_latitude'])
df['ocorrencia_longitude'] = pd.to_numeric(df['ocorrencia_longitude'])


Formatar as coordenadas

In [ ]:
estados_brasileiros = {
    "AC": (-9.115833, -67.784167),
    "AL": (-9.616667, -35.733333),
    "AP": (1.45, -51.75),
    "AM": (-3.100833, -60.025),
    "BA": (-12.971111, -38.477778),
    "CE": (-3.716667, -38.541667),
    "DF": (-15.794444, -47.882222),
    "ES": (-19.926111, -40.302778),
    "GO": (-15.981667, -49.253056),
    "MA": (-2.533333, -44.25),
    "MT": (-12.65, -55.45),
    "MS": (-20.448611, -54.627222),
    "MG": (-19.926111, -43.937222),
    "PA": (-1.458056, -48.483333),
    "PB": (-7.109444, -35.874444),
    "PR": (-25.428889, -51.521111),
    "PE": (-8.058333, -34.879167),
    "PI": (-7.216667, -41.766667),
    "RJ": (-22.908333, -43.196111),
    "RN": (-5.808889, -35.201111),
    "RS": (-30.033333, -51.216667),
    "RO": (-10.748889, -62.658056),
    "RR": (2.816667, -60.675),
    "SC": (-27.594444, -48.541667),
    "SP": (-23.550556, -46.633333),
    "SE": (-10.916667, -37.066667),
    "TO": (-10.183333, -48.325)
}


Atualizando a coluna da longitude, com os valores formadados.

In [ ]:
for index, row in df['ocorrencia_longitude'].items():

    if not np.isnan(row):
        log_base_10 = np.log10(abs(row))
        divisor = math.ceil(log_base_10) - 2
        if divisor != 0:
            df.loc[index, 'ocorrencia_longitude'] = row / (10**divisor)
        #print(index, row, df.loc[index, 'ocorrencia_longitude'])



Atualizando a coluna da latitude, com os valores formatados.

In [ ]:
for index, row in df['ocorrencia_latitude'].items():
    if not np.isnan(row):
        log_base_10 = np.log10(abs(row))
        divisor = math.ceil(log_base_10)
        if(row > 0):
            divisor -= 1
            df.loc[index, 'ocorrencia_latitude'] = row / (10 ** divisor)
        else:
            estado = df['ocorrencia_uf'].loc[index]  # Converte a Series em uma string

            divisor -= 2
            value = row/(10**divisor)
            if(abs(value - estados_brasileiros[estado][0]) > 10):
                value /= 10
            df.loc[index, 'ocorrencia_latitude'] = value


list = df.sort_values(by='ocorrencia_latitude')
print(list)


     ocorrencia_classificacao  ocorrencia_latitude  ocorrencia_longitude  \
4151                 ACIDENTE           -33.384722            -53.254444   
4152                 ACIDENTE           -33.384722            -53.254444   
4153                 ACIDENTE           -33.384722            -53.254444   
423                  ACIDENTE           -33.176944            -53.014167   
412                  ACIDENTE           -33.176944            -53.014167   
...                       ...                  ...                   ...   
4471                 ACIDENTE             5.084167            -60.345556   
4469                 ACIDENTE             5.084167            -60.345556   
4478                 ACIDENTE             5.084167            -60.345556   
4480                 ACIDENTE             5.084167            -60.345556   
5009          INCIDENTE GRAVE             8.713611             63.902778   

            ocorrencia_cidade ocorrencia_uf ocorrencia_pais  \
4151  SANTA VITÓRIA DO P

Mapa com as coordenadas

In [ ]:
import folium


# Extraindo as colunas de latitude e longitude
latitudes = df['ocorrencia_latitude']
longitudes = df['ocorrencia_longitude']

# Criando o mapa
mapa_mundo = folium.Map(location=[0, 0], zoom_start=2, tiles ='cartodbpositron')

# Adicionando marcadores
for lat, lon in zip(latitudes, longitudes):
  folium.Marker([lat, lon], popup=f"Incidente").add_to(mapa_mundo)


# Salvando o mapa
#mapa_mundo.save('mapa_mundo.html')

#mapa_mundo


Criando uma nova coluna com a data e a hora formatadas

In [ ]:
new_column = []
for dia, hora in zip(df['ocorrencia_dia'].astype(str), df['ocorrencia_hora'].astype(str)):
    dia = dia.split(' ')[0]
    hora = hora.split(' ')[0]
    new_column.append(dia + ' ' + hora)

df['ocorrencia_datahora'] = pd.to_datetime(new_column)
print(df['ocorrencia_datahora'])

del df['ocorrencia_hora']
del df['ocorrencia_dia']
del df['divulgacao_dia_publicacao']
del df['recomendacao_dia_assinatura']
del df['recomendacao_dia_encaminhamento']
del df['recomendacao_dia_feedback']


0      2022-04-14 17:45:00
1      2022-03-24 19:30:00
2      2022-03-24 19:30:00
3      2022-03-24 19:30:00
4      2022-03-24 19:30:00
               ...        
7562   2013-01-05 14:30:00
7563   2013-01-05 14:30:00
7564   2013-01-05 14:30:00
7565   2013-01-21 14:48:00
7566   2013-01-21 14:48:00
Name: ocorrencia_datahora, Length: 6889, dtype: datetime64[ns]


In [ ]:
print(df['aeronave_ano_fabricacao'].info())
print(df['aeronave_ano_fabricacao'])
coluna = df['aeronave_ano_fabricacao']
df = df.dropna(subset=['aeronave_ano_fabricacao'])

#Se o valor que estiver naquela linha da coluna tiver 5 digitos, divide por 10

def isCorrect(x):
  if(x >= 10000):
    return x//10

  return x

df['aeronave_ano_fabricacao'] =  coluna.apply(isCorrect)


<class 'pandas.core.series.Series'>
Index: 6842 entries, 0 to 7566
Series name: aeronave_ano_fabricacao
Non-Null Count  Dtype
--------------  -----
6842 non-null   int64
dtypes: int64(1)
memory usage: 106.9 KB
None
0       1980
1       1980
2       1980
3       1980
4       1980
        ... 
7562    2012
7563    2012
7564    2012
7565    2008
7566    2008
Name: aeronave_ano_fabricacao, Length: 6842, dtype: int64


In [ ]:
print(df.head())
print(df.shape)
print(df.isnull().sum())

  ocorrencia_classificacao  ocorrencia_latitude  ocorrencia_longitude  \
0          INCIDENTE GRAVE            -9.868333            -67.898056   
1                 ACIDENTE           -12.204167            -56.237778   
2                 ACIDENTE           -12.204167            -56.237778   
3                 ACIDENTE           -12.204167            -56.237778   
4                 ACIDENTE           -12.204167            -56.237778   

   ocorrencia_cidade ocorrencia_uf ocorrencia_pais ocorrencia_aerodromo  \
0         RIO BRANCO            AC          BRASIL                 SBRB   
1  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   
2  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   
3  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   
4  IPIRANGA DO NORTE            MT          BRASIL    Fora de aerodromo   

  investigacao_aeronave_liberada investigacao_status  \
0                            SIM          FINALIZADA  

### Remover as colunas irrelevantes para a ánalise

In [ ]:
del df['aeronave_matricula']
del df['recomendacao_status']
del df['recomendacao_numero']
del df['recomendacao_conteudo']
del df['divulgacao_relatorio_numero']


In [ ]:
print(df.shape)


(6842, 43)
